In [7]:
!python3.8 -m pip install .

Defaulting to user installation because normal site-packages is not writeable
Processing /home/j/Área de Trabalho/EstudosUnicamp/evolvepy
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for evolvepy: filename=evolvepy-0.0.0-py3-none-any.whl size=31213 sha256=64e5c683ea9aee1c63a001da2c0f1f4ab1b2d7e0c826df88e5026c6a9ec2c067
  Stored in directory: /tmp/pip-ephem-wheel-cache-rn8kn0_5/wheels/d3/c0/bb/a37f37518252ba78128e8808b5e9cc099a940b424005cef69b
Successfully built evolvepy


In [8]:
import evolvepy as ep
import numpy as np
import matplotlib.pyplot as plt

# Generator

Here we will define the generator for the problem

first, we need to define a descriptor with a boolean (true/false) chromossome

In [9]:
# here we have individual with 1 chromossome with type boolean. There is no need t define the chromossome varition range
descriptor = ep.generator.Descriptor(6, types = bool)

In [10]:
print(descriptor.dtype)

[('chr0', '?', (6,))]


In [11]:
from evolvepy import generator

# we will have populations with 50 individuals defined in the snippet above
n_individual = 50

# here we create the first layer i.e. generate the initial population
first_layer = generator.Layer()

In [12]:
from evolvepy.generator import CombineLayer
from evolvepy.generator.mutation import BinaryMutationLayer, bit_mutation
from evolvepy.generator.crossover import one_point
from evolvepy.generator.selection import tournament

combine = CombineLayer(tournament, one_point)
mutation = BinaryMutationLayer(bit_mutation, 0.8, 0.4, (0.0, 1.0))

gen = generator.Generator(descriptor=descriptor, first_layer=first_layer, last_layer=mutation)

# Run and Results

Let's evolve our population and see the results

In [13]:
evolver = ep.Evolver(gen, cache, n_individual)
hist, last_population = evolver.evolve(50)

NameError: name 'cache' is not defined

In [ ]:
plt.plot(np.max(hist, axis=1))
plt.plot(np.median(hist, axis=1))

plt.legend(["Best", "Median"])
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.title("Evolution History")
plt.show()

By looking at fitness history, we can see how the best individual improved over time, with the rest of the population following along.

![Evolution History](CarPIDControl-Figure1.jpg)

In [ ]:
_, bins, _ = plt.hist(hist[-1], histtype="step")
plt.hist(hist[0], bins, histtype="step")

plt.legend(["First generation", "Last generation"])
plt.xlabel("Generation")
plt.ylabel("Individuals")
plt.title("Fitness Histogram")
plt.show()

The fitness histogram of the first and last generations shows that the population was converging to a better region as the generations passed.

![Evolution History](CarPIDControl-Figure2.jpg)

We can even save a video of our best individual:

In [ ]:
best = last_population[np.argmax(hist[-1])]

test_evaluator = CarRacingEvaluator(save=True)
test_evaluator(best.ravel())

A previously recorded result can be seen in the video below (click on the image to go to the video).

[![Evolution History](CarPIDControl-Figure3.png)](https://www.youtube.com/watch?v=7NaK_CaXaeo)

It is important to emphasize that the performance of the optimization depends a lot on the way the evaluation is defined. Comparing an evolved population as it is on this notebook with an evolved one with only 3 ratings and only 1 generation cache, we can see how noise hinders the evolution process:

![Run Comparison](CarPIDControl-Figure4.png)

Finally, a complete log of all data can be seen in [Weights and Biases](https://wandb.ai/eltoncn/EvolvePy%20Example/runs/2880i6ay?workspace=user-eltoncn).